<a href="https://colab.research.google.com/github/DasariRishikesh/Compression_NeuralNets/blob/main/MNIST_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            # Compute scaled distillation loss from https://arxiv.org/abs/1503.02531
            # The magnitudes of the gradients produced by the soft targets scale
            # as 1/T^2, multiply them by T^2 when using both hard and soft targets.
            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

In [ ]:
# Create the teacher
teacher = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(512, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="teacher",
)

# Create the student
student = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(16, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="student",
)

# Clone student for later comparison
student_scratch = keras.models.clone_model(student)

In [ ]:
# Prepare the train and test dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalize data
x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 28, 28, 1))

x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 28, 28, 1))

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
# Train teacher as usual
teacher.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate teacher on data.
teacher.fit(x_train, y_train, epochs=5)
teacher.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 23s 7ms/step - loss: 0.1433 - sparse_categorical_accuracy: 0.9555
Epoch 2/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0924 - sparse_categorical_accuracy: 0.9729
Epoch 3/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0809 - sparse_categorical_accuracy: 0.9760
Epoch 4/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0767 - sparse_categorical_accuracy: 0.9783
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - loss: 0.0894 - sparse_categorical_accuracy: 0.9778


[0.08936827629804611, 0.9778000116348267]

In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - sparse_categorical_accuracy: 0.9096 - student_loss: 0.4169 - distillation_loss: 8.3386
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - sparse_categorical_accuracy: 0.9685 - student_loss: 0.1296 - distillation_loss: 2.1223
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - sparse_categorical_accuracy: 0.9740 - student_loss: 0.0998 - distillation_loss: 1.3815
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - sparse_categorical_accuracy: 0.9766 - student_loss: 0.0899 - distillation_loss: 1.1070
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - sparse_categorical_accuracy: 0.9750 - student_loss: 0.0997


[0.9750000238418579, 0.13092361390590668]

In [ ]:
# Train student as done usually
student_scratch.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
student_scratch.fit(x_train, y_train, epochs=5)
student_scratch.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2231 - sparse_categorical_accuracy: 0.9344
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0861 - sparse_categorical_accuracy: 0.9736
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0700 - sparse_categorical_accuracy: 0.9783
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0606 - sparse_categorical_accuracy: 0.9813
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0578 - sparse_categorical_accuracy: 0.9823


[0.057777076959609985, 0.9822999835014343]

In [ ]:
!pip install -q tensorflow-addons
!pip install -q tensorflow-model-optimization

     |████████████████████████████████| 1.1 MB 4.9 MB/s 
     |████████████████████████████████| 238 kB 5.1 MB/s 


In [ ]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = x_train.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(teacher, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "teacher"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d   (None, 14, 14, 256)      4866      
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_leaky_r  (None, 14, 14, 256)      1         
 e_lu (PruneLowMagnitude)                                        
                                                                 
 prune_low_magnitude_max_poo  (None, 14, 14, 256)      1         
 ling2d (PruneLowMagnitude)                                      
                                                                 
 prune_low_magnitude_conv2d_  (None, 7, 7, 512)        2359810   
 1 (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_flatten  (None, 25088)            1   

In [ ]:
_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(teacher, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Saved baseline model to: /tmp/tmphm9x9qur.h5


In [ ]:
 logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(x_train, y_train,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/2
  6/422 [..............................] - ETA: 9s - loss: 0.0601 - accuracy: 0.9792

422/422 [==============================] - 16s 25ms/step - loss: 0.0319 - accuracy: 0.9900 - val_loss: 0.0329 - val_accuracy: 0.9907
Epoch 2/2
422/422 [==============================] - 10s 24ms/step - loss: 0.0225 - accuracy: 0.9931 - val_loss: 0.0259 - val_accuracy: 0.9938


In [ ]:
teacher.evaluate(x_test,y_test)

313/313 [==============================] - 2s 5ms/step - loss: 0.0432 - sparse_categorical_accuracy: 0.9863


[0.043161600828170776, 0.986299991607666]

In [ ]:
distiller.evaluate(x_test,y_test)

313/313 [==============================] - 1s 3ms/step - sparse_categorical_accuracy: 0.9750 - student_loss: 0.0997


[0.9750000238418579, 0.13092361390590668]

In [ ]:
student_scratch.evaluate(x_test,y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.0578 - sparse_categorical_accuracy: 0.9823


[0.057777076959609985, 0.9822999835014343]

In [ ]:
model_for_pruning.evaluate(x_test,y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.0432 - accuracy: 0.9863


[0.043161600828170776, 0.986299991607666]